In [1]:
import numpy as np
import scipy.signal as signal
import pandas as pd
from wfdb import processing
import matplotlib.pyplot as plt
%matplotlib inline
import sys
import os
from hrvanalysis import remove_outliers, remove_ectopic_beats, interpolate_nan_values

In [2]:
class ECGSignal():
    def __init__(self, ecg_signal, Fs=1000, lowcut=0.3, highcut=60, resample_fs=128):
        self.ecg_signal = ecg_signal
        self.Fs = Fs
        self.lowcut = lowcut
        self.highcut = highcut
        self.resample_fs = resample_fs

        self.filtered_ecg_signal = self.apply_filter_to_signal()
        self.resampled_ecg_signal = signal.resample(self.filtered_ecg_signal, (len(self.ecg_signal) // self.Fs) * self.resample_fs)

        self.conf = processing.XQRS.Conf(ref_period=0.4)
        self.xqrs = processing.XQRS(sig=ecg_signal, fs=Fs, conf=self.conf)
        self.xqrs.detect(learn=False)
        peaks = self.xqrs.qrs_inds / Fs
        self.RRI = np.diff(peaks)

    def get_rri(self):
        return self.RRI

    def butter_bandpass(self, order=4):
    # to gain paramaters of butterworth bandpass filter
        nyq = 0.5 * self.Fs
        low = self.lowcut / nyq
        high = self.highcut / nyq
        b, a = signal.butter(order, [low, high], btype='band')
        return b, a

    def apply_filter_to_signal(self, order=4):
    # apply butterworth bandpass filter to the signal
        b, a = self.butter_bandpass(order)
        filtered = signal.filtfilt(b, a, self.ecg_signal)
        return filtered

In [ ]:
def csv_to_signal(filepath, header=None, skiprows=13):
    ecg_df = pd.read_table(filepath, header=header, skiprows=skiprows)[0]
    ecg_signal = ecg_df.values

    return ecg_signal

In [ ]:
def preprocess_hrvanalysis(rri, low_rri=300, high_rri=2000, interpolation_method="linear", remove_method="malik"):
    rri_int = list(map(int, rri * 1000))

    # This remove outliers from signal
    rr_intervals_without_outliers = remove_outliers(rr_intervals=rri_int,                                                     low_rri=low_rri, high_rri=high_rri)

    # This replace outliers nan values with linear interpolation
    interpolated_rr_intervals = interpolate_nan_values(rr_intervals=rr_intervals_without_outliers, interpolation_method=interpolation_method)

    # This remove ectopic beats from signal
    nn_intervals_list = remove_ectopic_beats(rr_intervals=interpolated_rr_intervals, method=remove_method)
    # This replace ectopic beats nan values with linear interpolation
    interpolated_nn_intervals = interpolate_nan_values(rr_intervals=nn_intervals_list)

    return rri_int, interpolated_rr_intervals, nn_intervals_list, interpolated_rr_intervals

In [ ]:
def calc_hrv_features(nn_intervals):
    time_domain_features = get_time_domain_features(nn_intervals)
    frequency_features = get_frequency_domain_features(nn_intervals)

    return time_domain_features, frequency_features)

In [ ]:
filepath = '../data/WAV_00000010.txt'

Fs = 1000  # [Hz]

ecg_signal = ECGSignal(csv_to_signal(filepath))
rri = ecg_signal.get_rri()

rri_int, interpolated_rr_intervals, nn_intervals_list, interpolated_rr_intervals = preprocess_hrvanalysis(rri)


window = 50 * 1000
step = 10 * 1000
window_num = (sum(rri_int) - window) // step + 1
for w in range(window_num):
    start = w * step
    end = window + w * step
    windowed_rri = nn_intervals_list[]